test

In [1]:
import pandas as pd
import boto3
import io

import spacy

nlp = spacy.load('en')

ModuleNotFoundError: No module named 'boto3'

In [ ]:
# Set up packages for loading in data
client = boto3.client('s3') #low-level functional API

resource = boto3.resource('s3') #high-level object-oriented API

In [ ]:
# Load in training data labels
obj = client.get_object(Bucket='thinkful-capstone', Key='training_variants')
stream = io.BytesIO(obj['Body'].read())
training_variants = pd.read_csv(stream)

In [ ]:
print(training_variants.head())
print(training_variants.shape)

In [ ]:
# Load in training data text articles
obj = client.get_object(Bucket="thinkful-capstone",Key="training_text")
raw_training_text = obj["Body"].read()

In [ ]:
# Split text file into list of documents
training_list = raw_training_text.split('||')
training_list = training_list[2:]
print(len(training_list))

In [ ]:
# Load training text into dataframe
texts_df = pd.DataFrame(training_list, columns = ['text'])

# Merge text dataframe with labels dataframe
train = pd.concat([training_variants, texts_df], axis=1)
print(train.head())

Now that the data is loaded into a dataframe, let's do some preliminary data cleaning and exploration.

In [ ]:
print(train['Class'].value_counts())

It looks like we are dealing with significant class imbalance here. Class 7 has 953 datapoints, while Class 8 has only 19. Because the labels have been anonymized, we cannot draw any insight about what these classes might signify. 
# We may wish to oversample the lesser represented classes at a later time point.

In [ ]:
train['spacy_tokens'] = train['text'].apply(lambda x: nlp(x))

def lemmatize(x):
    return [token.lemma_ for token in x
                      if not token.is_stop
                      and not token.is_punct
                      and not token.lemma_ == "-PRON-"
                      and not token.lemma_ == " " or "  "
    ]

train['lemmas'] = train['spacy_tokens'].apply(lambda x: lemmatize(x))
print(train.head())